# Kiểm soát chất lượng

## 1. Giới thiệu
Công nghệ giải trình tự RNA đơn bào (single-cell RNA sequencing, scRNA-seq) đã mở ra một phương pháp nghiên cứu mang tính đột phá, cho phép các nhà khoa học phân tích chi tiết sự đa dạng và biến động của các tế bào riêng lẻ trong hệ thống sinh học phức tạp. Phương pháp này đã làm thay đổi căn bản hiểu biết của chúng ta về biểu hiện gen ở cấp độ đơn bào, từ đó làm sáng tỏ các cơ chế điều hòa phân tử trong các trạng thái sinh lý và bệnh lý khác nhau.

Tuy nhiên, một thách thức then chốt trong việc ứng dụng công nghệ này nằm ở việc xử lý nhiễu kỹ thuật trong dữ liệu thu được. Đặc biệt, do việc giải trình tự được thực hiện trên một lượng rất nhỏ mRNA, hiện tượng "dropout" - tức là sự mất mát thông tin của các bản sao phiên mã trong quá trình phiên mã ngược - thường xuyên xảy ra và có thể làm sai lệch đáng kể kết quả phân tích định lượng. Ngoài ra, khi so sánh biểu hiện gen giữa các tế bào cùng type, người ta thường quan sát thấy những dao động ngẫu nhiên đáng kể, có thể là kết quả của các hiện tượng bùng phát phiên mã (transcriptional bursting) hoặc biến động tự nhiên trong quá trình điều hòa gen.

Do đó, việc thực hiện kiểm soát chất lượng nghiêm ngặt và loại bỏ các yếu tố nhiễu kỹ thuật trong dữ liệu scRNA-seq là một bước không thể thiếu, đóng vai trò quyết định đến độ tin cậy của các kết luận khoa học thu được.


<img src="plots/quality_control.jpeg" width=1200  />

*Hình 1. Dữ liệu RNA đơn bào có thể chứa các tế bào chất lượng kém, RNA tự do và tế bào kép. Việc kiểm soát chất lượng nhằm loại bỏ và điều chỉnh những yếu tố này để đạt được một bộ dữ liệu chất lượng cao, trong đó mỗi đơn vị dữ liệu là một tế bào đơn nguyên vẹn.*

Trong phân tích dữ liệu RNA đơn bào, một giả định nền tảng là mỗi điểm dữ liệu (observation) phản ánh chính xác thông tin phiên mã từ một tế bào đơn lẻ còn nguyên vẹn. Tuy nhiên, trên thực tế, tính toàn vẹn của giả định này thường bị ảnh hưởng bởi nhiều yếu tố kỹ thuật và sinh học. Cụ thể, sự hiện diện của các tế bào suy thoái, contamination từ RNA ngoại bào tự do, hay hiện tượng "doublet" - khi hai hay nhiều tế bào được giải trình tự đồng thời trong cùng một phản ứng - có thể làm sai lệch nghiêm trọng kết quả phân tích.

Chương này sẽ trình bày chi tiết các phương pháp và chiến lược để nhận diện, đánh giá và loại bỏ có hệ thống những yếu tố nhiễu này, nhằm xây dựng một tập dữ liệu có độ tin cậy cao, đảm bảo tính chính xác cho các phân tích downstream tiếp theo.

## 2.2. Lọc bỏ các tế bào chất lượng thấp


Bước then chốt đầu tiên trong quy trình kiểm soát chất lượng là việc loại bỏ có chọn lọc các tế bào không đạt tiêu chuẩn phân tích. Các dấu hiệu đặc trưng của tế bào suy thoái bao gồm: số lượng gene được phát hiện thấp bất thường, tổng số count RNA thấp, và đặc biệt là tỷ lệ phần trăm RNA ty thể (được đánh dấu bởi tiền tố mt-/MT- trong nomenclature) cao - một chỉ dấu điển hình của hiện tượng vỡ màng tế bào và tiến trình chết tế bào. Do những tế bào này có thể gây nhiễu đáng kể cho các phân tích downstream, việc loại bỏ chúng là cần thiết thông qua việc thiết lập các ngưỡng kiểm soát chất lượng phù hợp.

Ba thông số định lượng được sử dụng phổ biến trong kiểm soát chất lượng bao gồm:

- Tổng số count RNA trên mỗi barcode/tế bào
- Số lượng gene được phát hiện trên mỗi barcode/tế bào
- Tỷ lệ phần trăm count từ các gene ty thể trên mỗi barcode/tế bào

Các thông số này được sử dụng như những chỉ báo về tình trạng tế bào. Đặc biệt, tế bào suy thoái thường biểu hiện mô thức đặc trưng: số lượng count và gene thấp, đồng thời có tỷ lệ RNA ty thể cao do sự rò rỉ của RNA tế bào chất qua màng tế bào đã bị tổn thương.
Chiến lược tối ưu trong thiết lập ngưỡng
Việc phân tích đồng thời cả ba thông số này là vô cùng quan trọng để tránh những kết luận sai lệch. Ví dụ, một tỷ lệ RNA ty thể cao không nhất thiết chỉ ra tế bào đang suy thoái - nó có thể phản ánh hoạt động hô hấp tế bào mạnh mẽ. Tương tự, số lượng count RNA cao hoặc thấp có thể liên quan đến kích thước tế bào hoặc trạng thái hoạt động sinh học đặc thù.

Do đó, phương pháp tối ưu là:

- Xem xét tổng hợp nhiều thông số khi thiết lập ngưỡng cho từng tiêu chí
- Áp dụng chiến lược bảo thủ để tránh loại bỏ nhầm các quần thể tế bào có giá trị hoặc các type tế bào hiếm
- Duy trì tính linh hoạt trong việc thiết lập ngưỡng dựa trên đặc điểm sinh học của mẫu nghiên cứu

Bạn đọc có thể thực hành với trang sau: 

[https://www.sc-best-practices.org/preprocessing_visualization/quality_control.html#filtering-low-quality-cells](https://www.sc-best-practices.org/preprocessing_visualization/quality_control.html#filtering-low-quality-cells)

## 3. Hiệu chỉnh RNA tự do

Trong phương pháp giải trình tự RNA đơn bào dựa trên công nghệ vi giọt (droplet-based scRNA-seq), một hiện tượng kỹ thuật đáng quan tâm là sự hiện diện của RNA tự do trong dung dịch phản ứng. Cụ thể, khi các tế bào được phân tách vào từng vi giọt riêng biệt, RNA tự do từ môi trường - còn được gọi là "ambient RNA" hay "RNA soup" - cũng không tránh khỏi việc được đồng phân phối vào các vi giọt này. Hệ quả là tín hiệu RNA thu được từ mỗi vi giọt không chỉ phản ánh transcriptome của tế bào mục tiêu mà còn chứa đựng thông tin từ các nguồn RNA ngoại lai.

Công nghệ scRNA-seq sử dụng hệ thống "unique molecular identifiers" (UMI) để định lượng chính xác số lượng phân tử RNA cho mỗi gene trong từng tế bào. Tuy nhiên, giả định nền tảng rằng mỗi vi giọt chỉ chứa thông tin từ một tế bào đơn lẻ có thể bị vi phạm bởi nhiều yếu tố:

- Hiện tượng doublet: Khi hai hay nhiều tế bào cùng hiện diện trong một vi giọt
- Vi giọt rỗng: Chỉ chứa RNA tự do mà không có tế bào
- Contamination từ RNA ngoại bào (cfRNA): RNA tự do có nguồn gốc từ tế bào bị phân hủy trong quá trình chuẩn bị mẫu

RNA tự do này - thường được gọi là "RNA soup" - chủ yếu xuất phát từ quá trình phân hủy tự nhiên của tế bào trong quá trình xử lý mẫu, và có thể tạo ra tín hiệu nhiễu đáng kể trong dữ liệu thu được.

<img src="plots/ambient_rna.jpeg" width=1200  />

*Hình 2: Trong các công nghệ giải trình tự dựa trên droplet, các vi giọt có thể chứa RNA ngoại bào (cfRNA) hoặc xảy ra hiện tượng doublet. cfRNA trong môi trường sẽ được gắn mã barcode và giải trình tự cùng với mRNA từ tế bào mục tiêu, dẫn đến sự nhiễm tạp trong kết quả định lượng RNA.*

Sự hiện diện của RNA tự do trong môi trường có thể dẫn đến những ước lượng sai lệch về mức độ biểu hiện gene. RNA tự do này tạo ra một dạng nhiễu nền (background contamination) cần được xử lý một cách thận trọng và có hệ thống. Việc hiệu chỉnh ảnh hưởng của RNA tự do trong dữ liệu scRNA-seq là một bước không thể thiếu, bởi nó có thể ảnh hưởng nghiêm trọng đến độ tin cậy của các phân tích downstream.

Một điểm quan trọng cần lưu ý là thành phần và mức độ của RNA soup không đồng nhất giữa các mẫu sinh học. Thành phần của nó phụ thuộc vào:

- Đặc điểm của dung dịch đầu vào
- Mô hình biểu hiện gene của các loại tế bào có mặt trong mẫu
- Điều kiện xử lý và bảo quản mẫu

Để giải quyết vấn đề này, các công cụ phân tích chuyên biệt đã được phát triển:

- SoupX {cite}`qc:Young2020`: Một phương pháp tiên tiến giúp ước tính và loại bỏ ảnh hưởng của RNA môi trường
- DecontX {cite}`decontX_yang_decontamination_2020`: Công cụ phân tích cho phép định lượng thành phần RNA soup và hiệu chỉnh ma trận count dựa trên các mô hình thống kê

Các phương pháp này không chỉ giúp nhận diện và định lượng mức độ nhiễm RNA tự do mà còn cung cấp các giải pháp hiệu chỉnh để khôi phục tín hiệu thực của từng tế bào.


## 4. Phát hiện Doublet - vi giọt kép

Doublet là hiện tượng xuất hiện khi hai tế bào cùng được bắt giữ trong một vi giọt đơn lẻ, dẫn đến việc chúng chia sẻ cùng một barcode trong quá trình giải trình tự. Chính vì lý do này, trong lĩnh vực scRNA-seq, thuật ngữ "barcode" thường được ưu tiên sử dụng thay vì "cell" để tránh những giả định sai lầm về tính đơn nhất của tế bào. Doublet được phân thành hai loại chính:

- Doublet đồng hình (homotypic doublets): Hình thành từ hai tế bào cùng type (thường từ các cá thể khác nhau)
- Doublet dị hình (heterotypic doublets): Hình thành từ hai tế bào thuộc các type khác nhau

Doublet đồng hình thường không được coi là mối đe dọa nghiêm trọng trong phân tích, bởi chúng có thể được nhận diện thông qua các kỹ thuật như cell hashing hoặc phân tích SNP. Ngược lại, doublet dị hình đòi hỏi sự chú ý đặc biệt vì chúng có thể dẫn đến những sai lệch nghiêm trọng trong phân loại tế bào và các phân tích tiếp theo.

Các chiến lược phát hiện doublet hiện đại tập trung vào:

- Phân tích định lượng số lượng read và đặc điểm feature
- Tạo doublet nhân tạo làm cơ sở so sánh với dữ liệu thực nghiệm

Một nghiên cứu quan trọng của Xi và cộng sự {cite}`xi_benchmarking_2021` đã thực hiện đánh giá toàn diện 9 phương pháp phát hiện doublet khác nhau, cung cấp những hiểu biết quý giá về hiệu suất tính toán và độ chính xác của từng phương pháp.

Các bạn có thể đọc thêm hướng dẫn thực hành tại đây:

[https://www.sc-best-practices.org/preprocessing_visualization/quality_control.html#doublet-detection](https://www.sc-best-practices.org/preprocessing_visualization/quality_control.html#doublet-detection)

<img src="plots/doublet_detection.jpeg" width=1200  />

*Hình 3: Doublet là các giọt chứa nhiều hơn một tế bào. Các phương pháp phát hiện doublet hiện đại tạo ra các doublet nhân tạo bằng cách lấy mẫu ngẫu nhiên các cặp tế bào và tính trung bình profile biểu hiện gen của chúng. Các doublet nhân tạo này được phân tích cùng với các tế bào còn lại thông qua giảm chiều PCA. Phương pháp phát hiện doublet sau đó tính toán điểm số doublet dựa trên số lượng neighbors là doublet nhân tạo trong đồ thị KNN.*

## 4. Những điểm cần lưu ý

- Lọc tế bào chất lượng kém: Nên dựa trên độ lệch tuyệt đối trung bình (median absolute deviation - MAD) với ngưỡng rộng để tránh thiên vị đối với các quần thể nhỏ.
- Lọc dựa trên đặc trưng: Không cho thấy lợi ích cho các phân tích downstream.
- Phát hiện doublet: Có thể được thực hiện hiệu quả với các công cụ như [scDblFinderm](https://github.com/plger/scDblFinder) hay [Scrublet](https://github.com/swolock/scrublet)
- Phương pháp phát hiện doublet: Không nên được áp dụng trên dữ liệu scRNA-seq tổng hợp đại diện cho nhiều batch khác nhau, mà nên thực hiện trên từng dữ liệu một.
- Có thể sử dụng phần mềm tự động QC: [scAutoQC](https://github.com/Teichlab/sctk)

## 5. Thuật ngữ


| Thuật ngữ     | Dịch nghĩa | Ý nghĩa |
| ----------- | ----------- | ----------- |
| scRNAseq | Giải trình RNA tế bào đơn | giải trình tự RNA tế bào đơn (scRNA-seq) cho phép chúng ta phân tích biểu hiện gen của từng tế bào riêng lẻ trong một mẫu sinh học. |
| Transcriptomics | Hệ phiên mã | là một lĩnh vực nghiên cứu tập trung vào việc phân tích toàn bộ tập hợp các phân tử RNA được tạo ra từ một bộ gen cụ thể, tại một thời điểm nhất định và dưới một điều kiện nhất định. |
| transcript | Bản sao RNA | Transcript là bản sao RNA của một đoạn gen, mang thông tin để tổng hợp protein hoặc thực hiện các chức năng khác trong tế bào. |
| cell-free RNA (cfRNA) | RNA tự do | là các phân tử RNA tự do tồn tại trong môi trường ngoại bào, không nằm bên trong tế bào. |
| Ambient RNA | RNA môi trường | là các phân tử RNA tự do lơ lửng trong môi trường xung quanh các tế bào, không bị bao bọc bởi màng tế bào và không bao gồm Exosomes, ViRNA và các loại RNA khác |
| doublet | Vi giọt kép |  là một hiện tượng xảy ra khi hai tế bào khác nhau bị bắt giữ trong cùng một vi giọt |
| unique molecular identifiers (UMI) | Chỉ dấu phân tử duy nhất | là những đoạn mã ngắn, duy nhất được gắn vào mỗi phân tử RNA trước khi giải trình tự để phân biệt các phân tử RNA giống nha.u |
| count | Số lượng/lần đếm/lần xuất hiện | số lượng phân tử RNA duy nhất (UMI) của một gen cụ thể được phát hiện trong một tế bào, phản ánh mức độ biểu hiện của gen đó trong tế bào đó. |
| barcode | Mã vạch phân tử | là một đoạn trình tự nucleotide ngắn, duy nhất được gán cho mỗi tế bào hoặc mỗi vi giọt chứa tế bào để phân biệt chúng trong quá trình giải trình tự. |
| droplet | Vi giọt | là một khối chất lỏng cực nhỏ chứa một hoặc một vài tế bào đơn lẻ, được bao bọc bởi một lớp dầu để cách ly với các vi giọt khác. Mỗi vi giọt này hoạt động như một "ống nghiệm" siêu nhỏ, nơi các phản ứng sinh học để chuẩn bị thư viện giải trình tự RNA diễn ra độc lập. |
| soup | súp/hỗn hợp | sự nhiễm bẩn của mRNA tế bào tự do trong dung dịch đầu vào thường được gọi là "súp" (the soup), được tạo ra bởi sự phân hủy tế bào. |
| feature | đặc điểm/thuộc tính | là một biến số, một thuộc tính được trích xuất từ dữ liệu scRNAseq, như số lượng gen được biểu hiện, điểm số của một thành phần chính (principal component), hoặc một nhãn phân loại (ví dụ: loại tế bào). |
| cell hashing | Đánh dấu tế bào | là một kỹ thuật trong giải trình tự RNA tế bào đơn (scRNA-seq) cho phép chúng ta trộn nhiều mẫu tế bào khác nhau vào một ống nghiệm trước khi giải trình tự, đồng thời vẫn có thể phân biệt được các tế bào đến từ mẫu nào. |
| Barcode bead | Hạt mã vạch |  là những hạt nhỏ có gắn các đoạn trình tự DNA duy nhất (mã vạch) được sử dụng để đánh dấu và phân biệt các tế bào riêng lẻ trong quá trình giải trình tự RNA tế bào đơn (scRNA-seq). |
| Cell lysis | Sự phá vỡ tế bào |  là quá trình làm vỡ màng tế bào để giải phóng các thành phần bên trong, bao gồm cả RNA, ra môi trường xung quanh. Quá trình này nhằm mục đích thu thập RNA từ các tế bào để tiến hành giải trình tự và phân tích. |



## 6. Tham khảo

Bài này tổng hợp chủ yếu dựa trên [https://www.sc-best-practices.org/preprocessing_visualization/quality_control.html](https://www.sc-best-practices.org/preprocessing_visualization/quality_control.html)

```{bibliography}
:filter: docname in docnames
:labelprefix: at
```

## 7. Người đóng góp

### 7.1. Tác giả/dịch giả
- Duy Pham

### 7.2. Người review